In [ ]:
pip install git+https://github.com/tblume1992/statsforecast.git@feature/mfles

  Cloning https://github.com/tblume1992/statsforecast.git (to revision feature/mfles) to /tmp/pip-req-build-y37borof
  Running command git clone --filter=blob:none --quiet https://github.com/tblume1992/statsforecast.git /tmp/pip-req-build-y37borof
  Running command git checkout -b feature/mfles --track origin/feature/mfles
  Switched to a new branch 'feature/mfles'
  Branch 'feature/mfles' set up to track remote branch 'feature/mfles' from 'origin'.
  Resolved https://github.com/tblume1992/statsforecast.git to commit 5fc9e76fcd177256e6dcc617f231d3c3f02cdae2
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.7 MB/s eta 0:00

In [ ]:
pip install mfles

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from statsforecast.models import MSTL, AutoETS, AutoTBATS, AutoMFLES
from MFLES.Forecaster import MFLES as old_MFLES
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def lasso_nb(X, y, alpha, tol=0.001, maxiter=10000):
/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Hourly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Hourly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
mfles_smape = []
naive_smape = []
ets_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))
    mfles_model = AutoMFLES(
         season_length = [24, 168],
         test_size = 24,
         metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))


    ets_model = AutoETS(season_length=24, model='ZZZ')
    ets_model = ets_model.fit(y=y.values)
    fitted = ets_model.predict_in_sample()['fitted']
    predicted = ets_model.predict(h=len(y_test))['mean']
    ets_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.nanmean(mfles_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/414 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
New MFLES:17.066166156775495, Naive:42.86116392273107: 100%|██████████| 414/414 [27:59<00:00,  4.06s/it]

/b
Stats Forecast MFLES 11.73916434645399
ETS 17.191376329339505
Naive 43.002986836424824


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
naive_smape = []
mfles_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))
    mfles_model = AutoMFLES(
         season_length = [12],
         test_size = 12,
         metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.nanmean(mfles_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/48000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
New MFLES:17.191376329339505, Naive:15.256743936162751: 100%|██████████| 48000/48000 [3:25:50<00:00,  3.89it/s]


/b
Stats Forecast MFLES 12.917879309158353
Naive 15.256483534162772


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Weekly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Weekly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
naive_smape = []
mfles_smape = []
ets_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(mfles_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    mfles_model = AutoMFLES(
         season_length = [52],
         test_size = 26,
         metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))

    ets_model = AutoETS(season_length=52, model='ZZZ')
    ets_model = ets_model.fit(y=y.values)
    fitted = ets_model.predict_in_sample()['fitted']
    predicted = ets_model.predict(h=len(y_test))['mean']
    ets_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.mean(mfles_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/359 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
New MFLES:8.138029292920004, Naive:9.13930537684544: 100%|██████████| 359/359 [51:14<00:00,  8.56s/it]

/b
Stats Forecast MFLES 8.176053706408728
ETS 8.636611002006651
Naive 9.161286913982


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Quarterly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Quarterly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
naive_smape = []
mfles_smape = []
ets_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(mfles_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    mfles_model = AutoMFLES(
        season_length = [4],
        test_size = 4,
        metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))

    ets_model = AutoETS(season_length=4)
    ets_model = ets_model.fit(y=y.values)
    fitted = ets_model.predict_in_sample()['fitted']
    predicted = ets_model.predict(h=len(y_test))['mean']
    ets_smape.append(smape(y_test, predicted.clip(min=0)))

print('/b')
print(f'Stats Forecast MFLES {np.mean(mfles_smape)}')
print(f'ETS {np.nanmean(ets_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/24000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
New MFLES:10.72326902305219, Naive:11.610448045602293: 100%|██████████| 24000/24000 [2:04:37<00:00,  3.21it/s]


/b
Stats Forecast MFLES 10.723247041785744
ETS 10.245591381311593
Naive 11.610267157878681
